### ganache

### 1. 개발

In [13]:
%%writefile ../src/OrderCustomer.sol
pragma solidity >=0.5.0;

contract Customer {
    struct UserInfo {
        uint userId;
        string name;
        string phone;
        string addr;
        uint mileage;
        bool isCustomer;
    }
    mapping(address=>UserInfo) userMap;
    mapping(uint=>address) addressByUserId;
    mapping(address=>uint) balanceOfUser;
    
    function setUser(address addr, uint _userId, string memory _name, string memory _phone, string memory _addr) public {
        if (userMap[addressByUserId[_userId]].isCustomer) {
            revert("This account already exists");
        } else {
            UserInfo memory c = UserInfo(_userId, _name, _phone, _addr, 0, true);
            userMap[addr] = c;
            addressByUserId[_userId] = addr;
            balanceOfUser[addr] = addr.balance;
        }
    }
    function setMileage(address addr, uint _price, bool _status) public {
        uint m = _price/100;
        if (_status) {
            userMap[addr].mileage += m;
        } else {
            userMap[addr].mileage -= m;
        }
    }
    function getUserAddr(address addr) public view returns(string memory) {
        return userMap[addr].addr;
    }
    function withdraw(address addr, uint _price) public payable {
        require(balanceOfUser[addr] >= _price);
        balanceOfUser[addr] -= _price;
    }
    function refund(address addr, uint _price) public payable {
        balanceOfUser[addr] += _price;
    }
}

contract Order {
    struct OrderInfo {
        uint orderId;
        string item;
        uint n;    // number of items
        uint price;
        uint time;
        string status;
        bool isOrdered;
    }
    mapping(address=>OrderInfo) orderMap;
    mapping(uint=>address) addressByOrderId;
    Customer c;
    uint balanceOfOwner;
    uint orderCnt;
    constructor() public {
        c = new Customer();
        balanceOfOwner = 0;
        orderCnt = 0;
    }
    function setUser(uint _userId, string memory _name, string memory _phone, string memory _addr) public {
        c.setUser(msg.sender, _userId, _name, _phone, _addr);
    }
    function getUserAddr() public view returns(string memory) {
        return c.getUserAddr(msg.sender);
    }
    function order(uint _orderId, string memory _item, uint _n, uint _price) public payable {
        OrderInfo memory od = OrderInfo(_orderId, _item, _n, _price, now, "Ordered", true);
        orderMap[msg.sender] = od;
        addressByOrderId[_orderId] = msg.sender;
        c.setMileage(msg.sender, _price, true);
        orderCnt++;
        
        require(msg.value == _price);
        c.withdraw(msg.sender, _price);
        balanceOfOwner += _price;
    }
    function getOrderList() public view returns(uint, uint, uint) {
        return (orderCnt, balanceOfOwner, address(this).balance);
    }
    function getOrderListByAddr() public view returns(uint, string memory, string memory, string memory) {
        OrderInfo memory od = orderMap[msg.sender];
        return (od.orderId, od.item, od.status, c.getUserAddr(msg.sender));
    }
    function getOrderListById(uint _orderId) public view returns(uint, string memory, string memory, string memory) {
        OrderInfo memory od = orderMap[addressByOrderId[_orderId]];
        return (od.orderId, od.item, od.status, c.getUserAddr(addressByOrderId[_orderId]));
    }
    function refund(uint _orderId) public payable {
        require(addressByOrderId[_orderId] == msg.sender);
        OrderInfo memory od = orderMap[msg.sender];
        c.setMileage(msg.sender, od.price, false);
        orderCnt--;
        
        c.refund(msg.sender, od.price);
        balanceOfOwner -= od.price;
        msg.sender.transfer(od.price);
        
        orderMap[addressByOrderId[_orderId]].status = "Refunded";
        orderMap[addressByOrderId[_orderId]].price = 0;
    }
}

Overwriting ../src/OrderCustomer.sol


### 2. 컴파일

In [14]:
!solc --abi --bin --gas ../src/OrderCustomer.sol


======= ../src/OrderCustomer.sol:Customer =======
Gas estimation:
construction:
   575 + 541000 = 541575
external:
   getUserAddr(address):	infinite
   refund(address,uint256):	21162
   setMileage(address,uint256,bool):	21274
   setUser(address,uint256,string,string,string):	infinite
   withdraw(address,uint256):	22144
Binary:
608060405234801561001057600080fd5b50610a91806100206000396000f3fe60806040526004361061004a5760003560e01c80630c8493e31461004f578063410085df146100b6578063a15f554d14610104578063e8fc954e14610324578063f3fef3a3146103ee575b600080fd5b34801561005b57600080fd5b506100b46004803603606081101561007257600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291908035906020019092919080351515906020019092919050505061043c565b005b610102600480360360408110156100cc57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190803590602001909291905050506104fb565b005b34801561011057600080fd5b50610322600480360360a081101561012757600080fd5b81019080803573

### 3. 배포

In [71]:
%%writefile ../src/OrderCustomerDeploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"getOrderList","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getOrderListByAddr","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_orderId","type":"uint256"}],"name":"getOrderListById","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getUserAddr","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_orderId","type":"uint256"},{"internalType":"string","name":"_item","type":"string"},{"internalType":"uint256","name":"_n","type":"uint256"},{"internalType":"uint256","name":"_price","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_orderId","type":"uint256"}],"name":"refund","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_userId","type":"uint256"},{"internalType":"string","name":"_name","type":"string"},{"internalType":"string","name":"_phone","type":"string"},{"internalType":"string","name":"_addr","type":"string"}],"name":"setUser","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x"+"...";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    
    console.log("1-2 ==========================================================\n");
    await new web3.eth.Contract(_abi).deploy({data:_bin}).estimateGas(function (e, gas) {
        if (!e) console.log(">> order contract gas: "+gas);
    })
    console.log("\n==============================================================\n");
    
    console.log("Deploying the contract from "+accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:3000000}, function(err, transactionHash) {
            if (!err)
                console.log("hash: "+transactionHash);
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Overwriting ../src/OrderCustomerDeploy.js


In [75]:
!node ../src/OrderCustomerDeploy.js

1-2 ==========================================================

>> order contract gas: 2301563


Deploying the contract from 0xbc0bcd5aE1915dC56c7ff309707E5654566A2fe0
hash: 0x3c0139429316510cee1a8e1633320347f9e86d1b04a537c908f82c0ff97afe53
The contract deployed to: 0x1C12FEa92BAfD35C33b546CdF9B800E5Df2BFbfa


### 4. 사용

In [78]:
%%writefile ../src/OrderCustomerUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"getOrderList","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getOrderListByAddr","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_orderId","type":"uint256"}],"name":"getOrderListById","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"},{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getUserAddr","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_orderId","type":"uint256"},{"internalType":"string","name":"_item","type":"string"},{"internalType":"uint256","name":"_n","type":"uint256"},{"internalType":"uint256","name":"_price","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_orderId","type":"uint256"}],"name":"refund","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_userId","type":"uint256"},{"internalType":"string","name":"_name","type":"string"},{"internalType":"string","name":"_phone","type":"string"},{"internalType":"string","name":"_addr","type":"string"}],"name":"setUser","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _order = new web3.eth.Contract(_abi, "0x1C12FEa92BAfD35C33b546CdF9B800E5Df2BFbfa");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    // 1-1
    console.log("1-1 ==========================================================\n");
    console.log("accounts[0]: "+await web3.eth.getBalance(accounts[0]));
    console.log("accounts[1]: "+await web3.eth.getBalance(accounts[1]));
    console.log("accounts[2]: "+await web3.eth.getBalance(accounts[2])+"\n");
    await web3.eth.getBlockNumber(function (e, ret) {
        if (!e)
            console.log("block number: "+ret);
    });
    
    // 1-3
    console.log("\n1-3 ==========================================================\n\nsetUser");
    await _order.methods.setUser(111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul").send({from:accounts[0], gas:3000000})
    await _order.methods.setUser(112, "lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul").send({from:accounts[1], gas:3000000})
    await _order.methods.setUser(113, "lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul").send({from:accounts[2], gas:3000000})

    // 1-4
    console.log("\n1-4 ==========================================================\n");
    await _order.methods.getUserAddr().call({from:accounts[0]}).then(function(v) {console.log("accounts[0]: "+v)});
    await _order.methods.getUserAddr().call({from:accounts[1]}).then(function(v) {console.log("accounts[1]: "+v)});
    await _order.methods.getUserAddr().call({from:accounts[2]}).then(function(v) {console.log("accounts[2]: "+v)});
    
    // 1-5
    console.log("\n1-5 ==========================================================\n\norder");
    await _order.methods.order(555, "T-Shirt", 2, 1115).send({from:accounts[0], gas:3000000, value:1115});
    await _order.methods.order(556, "T-Shirt", 3, 1116).send({from:accounts[1], gas:3000000, value:1116});
    await _order.methods.order(557, "T-Shirt", 4, 1117).send({from:accounts[2], gas:3000000, value:1117});
    
    // 1-6
    console.log("\n1-6 ==========================================================\n");
    await _order.methods.getOrderList().call().then(function(v) {console.log("order count: "+v[0]+"\norder price: "+v[1]+"\ncontract balance: "+v[2])});
    
    // 1-7
    console.log("\n1-7 ==========================================================\n");
    await _order.methods.getOrderListByAddr().call({from:accounts[0]}).then(function(v) {console.log(v)});
    await _order.methods.getOrderListByAddr().call({from:accounts[1]}).then(function(v) {console.log(v)});
    await _order.methods.getOrderListByAddr().call({from:accounts[2]}).then(function(v) {console.log(v)});
    
    // 1-8
    console.log("\n1-8 ==========================================================\n");
    _order.methods.getOrderListById(556).call({from:accounts[1]}).then(function(v) {console.log(v)});
    
    // 1-9
    await _order.methods.refund(556).send({from:accounts[1], gas:3000000});
    
    // 1-10
    console.log("\n1-10 ==========================================================\n");
    _order.methods.getOrderListById(556).call({from:accounts[1]}).then(function(v) {console.log(v)});
    _order.methods.getOrderList().call().then(function(v) {console.log("\norder count: "+v[0]+"\norder price: "+v[1]+"\ncontract balance: "+v[2])});
}
doIt()

Overwriting ../src/OrderCustomerUse.js


In [77]:
!node ../src/OrderCustomerUse.js

1-1 ==========================================================

accounts[0]: 99668964379999998885
accounts[1]: 99990724940000000000
accounts[2]: 99991783199999998883

block number: 14

1-3 ==========================================================
setUser

1-4 ==========================================================

accounts[0]: 111 hongji-dong jongro-gu seoul
accounts[1]: 112 hongji-dong jongro-gu seoul
accounts[2]: 113 hongji-dong jongro-gu seoul

1-5 ==========================================================
order

1-6 ==========================================================

order count: 3
order price: 3348
contract balance: 3348

1-7 ==========================================================

Result {
  '0': '555',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '111 hongji-dong jongro-gu seoul'
}
Result {
  '0': '556',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '112 hongji-dong jongro-gu seoul'
}
Result {
  '0': '557',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '113 hongji-dong j